In [3]:
import pandas as pd
import os
#https://github.com/sean5446/icd9scraper

In [138]:
# For file in os.listdir('data/raw/'):
MAX_TABS = 3

for file in os.listdir("../data/raw/icd9"):
    if file.startswith(("cleaned", "dataframe")):
        continue
    
    with open(f"../data/raw/icd9/{file}", "r") as f:
        lines = f.readlines()
    
    with open(f"../data/raw/icd9/cleaned_{file}", "w") as f:
        for line in lines:
            tab_diff = MAX_TABS - line.count("\t")
            if tab_diff > 0:
                line = line.strip("\n") + "\t" * tab_diff + "\n"
            f.write(line)
            
    df = pd.read_csv(f"../data/raw/icd9/cleaned_{file}", sep="\t", header=None, usecols= [0,1,2], names=["chapter_codes", "block_codes", "category_codes"])
    if df.chapter_codes.nunique() == 1:
        if df.category_codes.isnull().all():
            df.category_codes = df.block_codes
        df[["chapter_codes", "block_codes"]] = df[["chapter_codes", "block_codes"]].ffill()
        df = df.dropna().reset_index(drop=True)
        df[["chapter_codes", "chapter_des"]] = df["chapter_codes"].str.split(" ", n=1, expand=True)
        df[["block_codes", "block_des"]] = df["block_codes"].str.split(" ", n=1, expand=True)
        df[["category_codes", "category_des"]] = df["category_codes"].str.split(" ", n=1, expand=True)
        df.to_csv(f"../data/raw/icd9/dataframe_{file.strip('.txt')}.csv", index=False)
    else:
        print("Problem", file)

In [145]:
df.shape

(123, 6)

In [7]:
df9 = pd.read_csv("../data/interim/icd9_codes_and_des.csv")
print(df9.shape)

df10 = pd.read_csv("../data/interim/icd10_codes_and_des.csv")
print(df10.shape)

(1234, 6)
(1949, 6)


In [94]:
df.chapter_codes.unique()

array(['001-139 Infectious And Parasitic Diseases'], dtype=object)

In [53]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re
import unicodedata

In [69]:
NHS_BASE = "https://www.nhs.uk/conditions/"

def get_symptoms_from_nhs(url: str) -> str | None:
    """
    Load symptoms from NHS

    Parameters
    ----------
    url : str
        URL to NHS page

    Returns
    -------
    str | None
        Either a string with symptoms, Message with error or None
    """
    res = requests.get(url)
    if res.status_code in range(200, 300):
        soup = BeautifulSoup(
            res.content,
            "lxml",
            parse_only=SoupStrainer("section"),
        )
        symptoms_list_raw = soup.findAll("h2")
        for item in soup.find_all("h2"):
            if "Symptoms" not in item.text:
                continue
            else:
                symptoms_list = item.find_next_siblings("ul")[0]
                text = ", ".join([li.text for li in symptoms_list.find_all("li")])
                text_cleaned = unicodedata.normalize("NFKD", text)
                return text_cleaned
        else:
            return f"No Value"
    return None

In [72]:
get_symptoms_from_nhs("https://www.nhs.uk/conditions/Histoplasmosis/")

In [5]:
import simple_icd_10_cm as cm


ModuleNotFoundError: No module named 'simple_icd_10_cm'

In [4]:
df = pd.read_csv("../data/interim/icd10_symptoms.csv")

In [6]:
df.shape

(1949, 7)

In [12]:
df1 = df.drop_duplicates().dropna().reset_index(drop=True)
df1.shape
df1.head()

,category_codes,category_codes_des,block_codes,block_codes_des,chapter_codes,chapter_codes_des,symptoms
0,A00,Cholera,A00-A09,Intestinal infectious diseases (A00-A09),1,Certain infectious and parasitic diseases (A00...,"Large amounts of watery diarrhea, vomiting, mu..."
1,A02,Other salmonella infections,A00-A09,Intestinal infectious diseases (A00-A09),1,Certain infectious and parasitic diseases (A00...,No Value
2,A03,Shigellosis,A00-A09,Intestinal infectious diseases (A00-A09),1,Certain infectious and parasitic diseases (A00...,"Diarrhea, fever, abdominal pain"
3,A06,Amebiasis,A00-A09,Intestinal infectious diseases (A00-A09),1,Certain infectious and parasitic diseases (A00...,"Bloody diarrhea, abdominal pain"
4,A15,Respiratory tuberculosis,A15-A19,Tuberculosis (A15-A19),1,Certain infectious and parasitic diseases (A00...,"Chronic cough, fever, cough with bloody mucus,..."


In [29]:
df1 = df.dropna()
df2 = df1[~(df1["symptoms"] == "No Value")]
df_grouped = df2.groupby("block_codes").agg({"symptoms": lambda x: list(x)})

In [31]:
df_grouped

,symptoms
block_codes,
A00-A09,"[Large amounts of watery diarrhea, vomiting, m..."
A15-A19,"[Chronic cough, fever, cough with bloody mucus..."
A20-A28,"[Fever, skin ulcer, large lymph nodes, Skin fo..."
A30-A49,"[Diarrhea, fever, headache, fever, cramped-up ..."
A50-A64,"[Firm, painless, non-itchy skin ulcer, Firm, p..."
...,...
R10-R19,"[Burning, stabbing, or squeezing sensation in ..."
R30-R39,"[Blood in the urine, Sudden onset: Inability t..."
R50-R69,"[fever; Hard, fixed, rapidly growing nodes, in..."
